In [1]:
import os
from dotenv import load_dotenv
from aocd import get_data

In [2]:
load_dotenv()
secret_key = os.getenv("SECRET_KEY")
os.environ['AOC_SESSION'] = secret_key

data = get_data(year=2023, day=10)

## Part 1
You make a quick sketch of all of the surface pipes you can see (your puzzle input).

The pipes are arranged in a two-dimensional grid of tiles:
```
| is a vertical pipe connecting north and south.
- is a horizontal pipe connecting east and west.
L is a 90-degree bend connecting north and east.
J is a 90-degree bend connecting north and west.
7 is a 90-degree bend connecting south and west.
F is a 90-degree bend connecting south and east.
. is ground; there is no pipe in this tile.
S is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.
```
Based on the acoustics of the animal's scurrying, you're confident the pipe that contains the animal is one large, continuous loop.

For example, here is a square loop of pipe:
```
.....
.F-7.
.|.|.
.L-J.
.....
```
If the animal had entered this loop in the northwest corner, the sketch would instead look like this:
```
.....
.S-7.
.|.|.
.L-J.
.....
```
In the above diagram, the S tile is still a 90-degree F bend: you can tell because of how the adjacent pipes connect to it.

Unfortunately, there are also many pipes that aren't connected to the loop! This sketch shows the same loop as above:
```
-L|F7
7S-7|
L|7||
-L-J|
L|-JF
```
In the above diagram, you can still figure out which pipes form the main loop: they're the ones connected to S, pipes those pipes connect to, pipes those pipes connect to, and so on. Every pipe in the main loop connects to its two neighbors (including S, which will have exactly two pipes connecting to it, and which is assumed to connect back to those two pipes).

Here is a sketch that contains a slightly more complex main loop:
```
..F7.
.FJ|.
SJ.L7
|F--J
LJ...
```
Here's the same example sketch with the extra, non-main-loop pipe tiles also shown:
```
7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ
```
If you want to get out ahead of the animal, you should find the tile in the loop that is farthest from the starting position. Because the animal is in the pipe, it doesn't make sense to measure this by direct distance. Instead, you need to find the tile that would take the longest number of steps along the loop to reach from the starting point - regardless of which way around the loop the animal went.

In the first example with the square loop:
```
.....
.S-7.
.|.|.
.L-J.
.....
```
You can count the distance each tile in the loop is from the starting point like this:
```
.....
.012.
.1.3.
.234.
.....
```
In this example, the farthest point from the start is 4 steps away.

Here's the more complex loop again:
```
..F7.
.FJ|.
SJ.L7
|F--J
LJ...
```
Here are the distances for each tile on that loop:
```
..45.
.236.
01.78
14567
23...
```
Find the single giant loop starting at S. How many steps along the loop does it take to get from the starting position to the point farthest from the starting position?



In [7]:
connect_left = '-LF'
connect_right = '-J7'
connect_up = '|7F'
connect_down = '|LJ'

directions = {
    'left': (0, -1, connect_left),   # Left
    'right': (0, 1, connect_right),   # Right
    'up': (-1, 0, connect_up),     # Up
    'down': (1, 0, connect_down)     # Down
}

pipes = {
    '|': ['up','down'],
    '-': ['left','right'],
    'L': ['up','right'],
    'J': ['left','up'],
    '7': ['left','down'],
    'F': ['right','down']
}

def find_s(data):
    for row_num,row in enumerate(data.splitlines()):
        for col_num,col in enumerate(data.splitlines()[row_num]):
            if data.splitlines()[row_num][col_num] == 'S':
                return (row_num, col_num)

def find_borders(data):
    height = len(data.splitlines())
    width = len(data.splitlines()[0])

    return height, width

def find_connections(data: str, xy: tuple, height: int, width: int):
    row_num, col_num = xy
    pipe = data.splitlines()[row_num][col_num]
    if pipe == 'S':
        possible_moves = ['up','down','left','right']
    else:
        possible_moves = pipes[pipe]
    connections = [] # list of tuples
    for direction in [x for x in directions if x in possible_moves]:
        r, c, chars = directions[direction]
        if data.splitlines()[row_num+r][col_num+c] in chars:
            connections.append((row_num+r, col_num+c))

    return connections

def find_loop(data: str, s: tuple, s_connections: list, height: int, width: int):
    # Create loop
    loop = set([s] + s_connections)
    # Create 'already checked'
    checked = set([s, s_connections[0]])

    # Loop through new connections
    while len(checked) < len(loop):
        for xy in [pos for pos in loop if pos not in checked]:
            connections = find_connections(data, xy, height, width)
            # Add found connections to loop
            loop |= set([coord for coord in connections if coord not in loop])
            # Add xy to checked
            checked.add(xy)

    return loop

def solve_part1(data):
    s = find_s(data)
    height, width = find_borders(data)
    s_connections = find_connections(data, s, height, width)
    loop = find_loop(data, s, s_connections, height, width)
    max_steps = len(loop) // 2
    return max_steps

In [8]:
solve_part1(data)

7102

## Part 2
Of course, it would be nice to have even more history included in your report. Surely it's safe to just extrapolate backwards as well, right?

For each history, repeat the process of finding differences until the sequence of differences is entirely zero. Then, rather than adding a zero to the end and filling in the next values of each previous sequence, you should instead add a zero to the beginning of your sequence of zeroes, then fill in new first values for each previous sequence.

In particular, here is what the third example history looks like when extrapolating back in time:
```
5  10  13  16  21  30  45
  5   3   3   5   9  15
   -2   0   2   4   6
      2   2   2   2
        0   0   0
```
Adding the new values on the left side of each sequence from bottom to top eventually reveals the new left-most history value: 5.

Doing this for the remaining example data above results in previous values of -3 for the first history and 0 for the second history. Adding all three new values together produces 2.

Analyze your OASIS report again, this time extrapolating the previous value for each history. What is the sum of these extrapolated values?

In [17]:
def get_loop_borders(loop):
    max_row = max(coord[0] for coord in loop)
    max_col = max(coord[1] for coord in loop)
    min_row = min(coord[0] for coord in loop)
    min_col = min(coord[1] for coord in loop)
    loop_borders = {
        'row': {'min': min_row, 'max': max_row},
        'col': {'min': min_col, 'max': max_col}
    }

    return loop_borders

def get_non_loop(loop, loop_borders):
    non_loop = set() # set of tuples
    for row in range(loop_borders['row']['min'], loop_borders['row']['max']+1):
        for col in range(loop_borders['col']['min'], loop_borders['col']['max']+1):
            if (row, col) not in loop:
                non_loop.add((row,col))

    return non_loop

def count_intersections(point, loop, data, width):
    row, col = point
    crossings = 0
    current_corner = None
    
    # Scan from point to right edge
    for x in range(col + 1, width):
        if (row, x) in loop:
            char = data.splitlines()[row][x]
            
            if char == '|':
                crossings += 1
            elif char == 'S':
                # Need to determine what type of pipe S represents
                # For now, treating S as '|' since we know it must be part of a valid loop
                crossings += 1
            elif char in 'FL7J':
                if current_corner is None:
                    current_corner = char
                else:
                    # Handle corner pairs
                    if (current_corner == 'F' and char == 'J') or \
                       (current_corner == 'L' and char == '7'):
                        # These pairs count as a crossing
                        crossings += 1
                    elif (current_corner == 'F' and char == '7') or \
                         (current_corner == 'L' and char == 'J'):
                        # These pairs don't count as a crossing
                        pass
                    # Reset for next pair
                    current_corner = None
    
    return crossings

def get_inside_loop(non_loop, loop, data, width):
    inside_loop = set() # set of tuples
    # Check all non-loop points
    for point in non_loop:
        intersections = count_intersections(point, loop, data, width)
        # If number of intersections is odd, it's inside loop
        if intersections % 2 > 0:
            inside_loop.add(point)
    
    return inside_loop

def solve_part2(data):
    s = find_s(data)
    height, width = find_borders(data)
    s_connections = find_connections(data, s, height, width)
    loop = find_loop(data, s, s_connections, height, width)
    loop_borders = get_loop_borders(loop)
    non_loop = get_non_loop(loop, loop_borders)
    inside_loop = get_inside_loop(non_loop, loop, data, width)

    return len(inside_loop)

In [18]:
solve_part2(data)

363